### Get relevant libraries \& packages

In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from qiskit import Aer, QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.tools.visualization import circuit_drawer, plot_histogram
from scipy.optimize import minimize
np.set_printoptions(precision=3, suppress=True)

In [11]:
%matplotlib inline

### Simple hamiltonian (mixing + cost), parameterized by one \gamma = beta = 0 

In [12]:
backend = Aer.get_backend('qasm_simulator')

In [13]:
gamma = 0
beta = 0
q = QuantumRegister(2, "q")
c = ClassicalRegister(2, "c")
circuit = QuantumCircuit(q, c)

for i in range(2):
    circuit.h(q[i])

circuit.cx(q[0],q[1])
circuit.rz(2*gamma,q[1])
circuit.cx(q[0],q[1])
circuit.rx(2*beta, q[0])
circuit.rx(2*beta, q[1])


circuit.measure(q, c)
shots = 500
job = execute(circuit, backend, shots=shots)

result = job.result().get_counts(circuit)

fig = circuit.draw('mpl')
#fig.savefig("../images/varCircs.png")

In [14]:
def evaluate_quantum_cost_hamiltonian(beta_gamma):
    
# extract the beta's and the gamma's from the single parameter np.array
    n = len(beta_gamma)//2
    beta  = beta_gamma[:n]
    gamma = beta_gamma[n:]

# Define a 'clean' circuit and initiate the qubits to a full superposition initial state
    q = QuantumRegister(n_qubits)
    c = ClassicalRegister(n_qubits)
    circuit = QuantumCircuit(q, c)
    
    for i in range(n_qubits):
        circuit.h(q[i])

    for i in range(p):
        # apply gamma's
        circuit.cx(q[0],q[1])
        circuit.rz(2*gamma[i],q[1])
        circuit.cx(q[0],q[1])
        # apply beta's
        circuit.rx(2*beta[i], q[0])
        circuit.rx(2*beta[i], q[1])
        
   
    circuit.measure(q, c)
    shots = 500
    job = execute(circuit, backend, shots=shots)

    result = job.result().get_counts(circuit)

# compute expectation
    H_expectation = 0
    for i in range(n_qubits):
        for j in range(n_qubits):
            multiplicity = result.get(str(i)+str(j), 0)
            H_expectation -= multiplicity*(2*i-1)*(2*j-1)

    H_expectation /= shots
    return np.real(H_expectation)

In [15]:
p = 2
n_qubits = 2
beta = np.random.uniform(0, 2*np.pi, p)
gamma = np.random.uniform(0, 2*np.pi, p)

result = minimize(evaluate_quantum_cost_hamiltonian, np.concatenate([beta, gamma]), method='COBYLA')
result

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.964
       x: [ 4.110e+00  4.201e+00  4.008e-01  5.963e+00]
    nfev: 47
   maxcv: 0.0

In [16]:
q = QuantumRegister(2)
c = ClassicalRegister(2)

beta  = result.x[:p]
gamma = result.x[p:]

#Create the initial state Ψ0  
n_qubits = 2
circuit = QuantumCircuit(q, c)
for i in range(n_qubits):
    circuit.h(q[i])    

#apply the evolution unitaries
for i in range(p):
    # apply gamma's
    circuit.cx(q[0],q[1])
    circuit.rz(-2*gamma[i],q[1])
    circuit.cx(q[0],q[1])
    # apply beta's
    circuit.rx(-2*beta[i], q[0])
    circuit.rx(-2*beta[i], q[1])
circuit.measure(q, c)

In [17]:
job = execute(circuit, backend, shots = 1000)
result = job.result().get_counts(circuit)
fig = plot_histogram(job.result().get_counts(circuit))
#fig.savefig("../images/varCircs_state.png")

In [18]:
fig.show()

C:\Users\spenc\AppData\Local\Temp\ipykernel_18128\89474557.py:1: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
